In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("../dataset/wind_engineered_all.csv")

In [4]:
df.head()

,Customer,Generator Capacity,datetime,label,CL,GC,GG
0,1,3.78,2010-07-01 00:00:00,1,1.075,0.125,0.0
1,1,3.78,2010-07-01 00:30:00,2,1.250,0.303,0.0
2,1,3.78,2010-07-01 01:00:00,3,1.244,0.471,0.0
3,1,3.78,2010-07-01 01:30:00,4,1.256,0.083,0.0
4,1,3.78,2010-07-01 02:00:00,5,0.744,0.121,0.0


In [5]:
filtered_df = df.drop(columns=["datetime"])
filtered_df.columns = ["location","gen_capacity","time_seq","CL","GC","GG"]

In [7]:
new_order = ["location","time_seq","gen_capacity","CL","GC","GG"]
filtered_df=filtered_df.reindex(columns=new_order)

In [8]:
filtered_df.head()

,location,time_seq,gen_capacity,CL,GC,GG
0,1,1,3.78,1.075,0.125,0.0
1,1,2,3.78,1.250,0.303,0.0
2,1,3,3.78,1.244,0.471,0.0
3,1,4,3.78,1.256,0.083,0.0
4,1,5,3.78,0.744,0.121,0.0


In [24]:
filtered_df.to_csv("../dataset/wind_engineering_edited.csv", index=False)

In [11]:
import sdv
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

In [12]:
metadata = SingleTableMetadata()

In [13]:
metadata.detect_from_dataframe(data=filtered_df)

In [14]:
python_dict = metadata.to_dict()

In [15]:
python_dict

{'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1',
 'columns': {'location': {'sdtype': 'numerical'},
  'time_seq': {'sdtype': 'numerical'},
  'gen_capacity': {'sdtype': 'numerical'},
  'CL': {'sdtype': 'numerical'},
  'GC': {'sdtype': 'numerical'},
  'GG': {'sdtype': 'numerical'}}}

In [16]:
metadata.validate()

In [17]:
synthesizer = CTGANSynthesizer(
    metadata,
    enforce_rounding=False,
    verbose=True
)

In [20]:
df_1 = filtered_df[filtered_df["location"]<11]

In [21]:
df_1.shape

(175200, 6)

In [22]:
synthesizer.fit(df_1)

Epoch 1, Loss G: -1.1574,Loss D: -0.1352
Epoch 2, Loss G: -1.4669,Loss D:  0.0431
Epoch 3, Loss G: -1.3401,Loss D: -0.1471
Epoch 4, Loss G: -0.5084,Loss D: -0.3538
Epoch 5, Loss G:  0.0284,Loss D: -0.4268
Epoch 6, Loss G:  0.3242,Loss D: -0.6037
Epoch 7, Loss G:  0.2996,Loss D: -0.5721
Epoch 8, Loss G:  0.2082,Loss D: -0.4750
Epoch 9, Loss G:  0.2214,Loss D: -0.5048
Epoch 10, Loss G:  0.2642,Loss D: -0.4979
Epoch 11, Loss G:  0.3562,Loss D: -0.5485
Epoch 12, Loss G:  0.1506,Loss D: -0.3892
Epoch 13, Loss G: -0.0487,Loss D: -0.3926
Epoch 14, Loss G: -0.1507,Loss D: -0.4210
Epoch 15, Loss G:  0.0572,Loss D: -0.0770
Epoch 16, Loss G: -0.0563,Loss D: -0.2202
Epoch 17, Loss G:  0.1322,Loss D: -0.3143
Epoch 18, Loss G: -0.1870,Loss D: -0.1961
Epoch 19, Loss G: -0.0893,Loss D: -0.2060
Epoch 20, Loss G: -0.2759,Loss D: -0.2091
Epoch 21, Loss G: -0.0024,Loss D: -0.2905
Epoch 22, Loss G: -0.1549,Loss D: -0.1493
Epoch 23, Loss G: -0.1990,Loss D: -0.1816
Epoch 24, Loss G: -0.1177,Loss D: -0.2559
E

In [23]:
synthesizer.save(
    filepath='../saved_model/final_model.pkl'
)